In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
%pip install -q -U keras-tuner
import kerastuner as kt

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_12266/2802655024.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
(img_train, label_train), (img_test, label_test) = (
    keras.datasets.fashion_mnist.load_data()
)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [5]:
tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="intro_to_kt",
)

/home/chan/code/vscode/tensor-cpp/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [7]:
tuner.search(
    img_train,
    label_train,
    epochs=10,
    validation_data=(img_test, label_test),
    callbacks=[ClearTrainingOutput()],
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(
    f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
"""
)

Trial 16 Complete [00h 00m 13s]
val_accuracy: 0.8140000104904175

Best val_accuracy So Far: 0.8270000219345093
Total elapsed time: 00h 03m 40s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
384               |384               |units
0.001             |0.001             |learning_rate
10                |4                 |tuner/epochs
4                 |2                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |1                 |tuner/round
0014              |0007              |tuner/trial_id

Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8275 - loss: 0.5422 - val_accuracy: 0.8286 - val_loss: 0.5158
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8388 - loss: 0.4819 - val_accuracy: 0.8321 - val_loss: 0.5208
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8406 - loss: 0.4712 - val_accuracy: 0.8347 - val_loss: 0.5161
Epoch 8/10
1875/1875 ━━━━

KeyboardInterrupt: 

In [8]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

NameError: name 'best_hps' is not defined